In [ ]:
#!/usr/bin/env python3
import os, sys
import numpy as np
import qutip as qt
import itertools

import scipy.linalg
import scipy.sparse
from scipy.integrate import solve_ivp
from scipy.optimize import curve_fit

import multiprocessing as mp

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.lines import Line2D

from correlator_methods import mat_zxy_to_pzm, vec_zxy_to_pzm, op_ln_val_ZXY, \
    get_deriv_op_vec, sandwich_deriv_op_vec, add_deriv_op_vecs, multiply_deriv_op_vecs, \
    deriv_op_vec_to_vals, deriv_vals_to_correlators

from time import time

np.set_printoptions(linewidth = 200)

In [ ]:
vecs_A, vecs_B = {}, {}
vecs_At_B, vecs_B_At = {}, {}
vecs_FPC = {}
corrs_A, corrs_At_B = {}, {}
corrs_TTC, corrs_TTC_exact = {}, {}
corrs_FPC, corrs_FPC_exact = {}, {}
all_dicts = ( vecs_A, vecs_B, vecs_At_B, vecs_B_At, vecs_FPC,
              corrs_A, corrs_At_B, corrs_TTC, corrs_FPC,
              corrs_TTC_exact, corrs_FPC_exact )

vecs_A_str = "vecs_A"
vecs_B_str = "vecs_B"
vecs_At_B_str = "vecs_At_B"
vecs_B_At_str = "vecs_B_At"
vecs_FPC_str = "vecs_FPC"
dec_0 = "dec_0"
dec_s = "dec_s"
NAT = "NAT"
OAT = "OAT"
TAT = "TAT"
TNT = "TNT"
TNB = "TNB"
all_methods = [ NAT, OAT, TAT, TNT, TNB ]

save_dir = "../data/multi_time/"
fig_dir = "../figures/multi_time/"
tmp_dir = "./tmp/"

In [ ]:
free_cores = 2 # number of cores to leave free when multiprocessing
order_cap = 20 # order limit for short-time correlator expansion
methods = [ OAT, TAT, TNT ]

N = 10**4 # number of spins
dec_val = N/100 # decoherence strength
if dec_val == int(dec_val): dec_val = int(dec_val)

def save_file_name(base_name, method, dec):
    return save_dir + f"{base_name}_{method}_{dec}_N{N}_O{order_cap}.txt"

time_steps = 200
max_time = 20 # maximum timulation time in units with N \chi = 1
times = np.linspace(0, max_time / N, time_steps+1)

ivp_tolerance = 1e-13 # relative error tolerance in numerical integrator

S = N/2
h_zxy = {}
h_zxy[NAT] = {}
h_zxy[OAT] = { (2,0,0) : 1 }
h_zxy[TAT] = { (2,0,0) : +1/3,
               (0,0,2) : -1/3 }
h_zxy[TNT] = { (2,0,0) : 1,
               (0,1,0) : S }
h_zxy[TNB] = { (2,0,0) : 1,
                (0,1,0) : -S }

init_state_X = (0,1,0)
init_state_nZ = (-1,0,0)
mat_X_to_nZ_zxy = -np.array([ [0,1,0], [1,0,0], [0,0,1] ])
mat_X_to_nZ_pzm = mat_zxy_to_pzm(mat_X_to_nZ_zxy)

dec_rates = { dec_0 : None,
              dec_s : (dec_val,)*3 }
dec_labels = { dec_0 : r"$\gamma_0=0$",
               dec_s : r"$\gamma_0=" + f"{dec_val}" + r"\chi$" }

dec_grid_map = { dec : jj for jj, dec in enumerate(dec_rates.keys()) }
for single_dict in all_dicts:
    for dec in dec_rates:
        if single_dict.get(dec) is None:
            single_dict[dec] = {}

A_zxy = { (0,1,0) : 1,
          (0,0,1) : 1j }
B_zxy = { (0,1,0) : 1,
          (0,0,1) : -1j }

h_pzm = { method : vec_zxy_to_pzm(h_zxy[method]) for method in methods }
A_pzm = { method : vec_zxy_to_pzm(A_zxy) for method in methods }
B_pzm = { method : vec_zxy_to_pzm(B_zxy) for method in methods }

B_init_val = 0
for op, val in B_pzm[methods[0]].items():
    ln_val, phase = op_ln_val_ZXY(op, N)
    B_init_val += np.exp(ln_val) * phase

In [ ]:
##########################################################################################
# exact few-qubit simulation methods
##########################################################################################
exact_simulation_cutoff_size = 6

def comm(A, B): return A @ B - B @ A
def acomm(A, B): return A @ B + B @ A
def flat_time_deriv(flat_op_mat, hamiltonian, dec_sets):
    op_mat = flat_op_mat.reshape((2**N,2**N))
    time_deriv = 1j * comm(hamiltonian, op_mat)
    for dec_set in dec_sets:
        time_deriv += sum( dec_op[0].conj().T @ op_mat @ dec_op[0]
                           - 1/2 * acomm(dec_op[1], op_mat)
                           for dec_op in dec_set )
    return time_deriv.flatten()

def op_at_time(op_0, time, hamiltonian, dec_sets):
    if time == 0: return op_0
    time_deriv = lambda time, flat_op : flat_time_deriv(flat_op, hamiltonian, dec_sets)
    flat_op_t = solve_ivp(time_deriv, (0,time), op_0.flatten(), t_eval = [time],
                          rtol = ivp_tolerance, atol = ivp_tolerance).y[:,0]
    return flat_op_t.reshape((2**N,2**N))

if N <= exact_simulation_cutoff_size:

    I2 = qt.qeye(2)
    up = qt.basis(2,0)
    dn = qt.basis(2,1)
    II = qt.tensor([I2]*N)
    ZZ = 0 * II

    init_state_X_vec = qt.tensor([ up + dn ]*N).data.toarray() / 2**(N/2)

    # individual spin operators
    s_z = []
    s_p = []
    s_m = []
    for jj in range(N):
        s_z.append(qt.tensor([ I2 ] * jj + [ qt.sigmaz()/2 ] + [ I2 ] * (N-jj-1)))
        s_p.append(qt.tensor([ I2 ] * jj + [ qt.sigmap() ] + [ I2 ] * (N-jj-1)))
        s_m.append(qt.tensor([ I2 ] * jj + [ qt.sigmam() ] + [ I2 ] * (N-jj-1)))

    # collective spin operators
    S_p = sum(s_p)
    S_m = sum(s_m)
    S_z = sum(s_z)
    S_x =   1/2 * ( S_p + S_m )
    S_y = -1j/2 * ( S_p - S_m )

    # convert vector of operators (in zxy format) to a matrix
    def op_vec_to_mat(vec):
        mat = ZZ
        for op, val in vec.items():
            mat += val * S_z**op[0] * S_x**op[1] * S_y**op[2]
        return mat.data.toarray()

    # hamiltonian
    H_mat = { method : op_vec_to_mat(h_zxy[method]) for method in methods }
    A_mat = op_vec_to_mat(A_zxy)
    B_mat = op_vec_to_mat(B_zxy)

    # decoherence vectors
    dec_sets = {}
    for name, dec in dec_rates.items():
        dec_sets[name] = []
        if name is dec_0: continue
        for rate, ops in [ ( dec[0][0], s_p ), ( dec[0][1], s_z ), ( dec[0][2], s_m ),
                           ( dec[1][0], [S_p] ), ( dec[1][1], [S_z] ), ( dec[1][2], [S_m] ) ]:
            if rate == 0: continue
            dec_sets[name].append([ ( np.sqrt(rate) * op.data.toarray(),
                                      rate * (op.dag()*op).data.toarray() )
                                    for op in ops ])

In [ ]:
def expectation_value(op_mat):
    return ( init_state_X_vec.conj().T @ op_mat @ init_state_X_vec )[0,0]

if N <= exact_simulation_cutoff_size:

    start = time()

    for method, dec in itertools.product(methods, dec_rates):
        print(method, dec)
        At_mats = [ op_at_time(A_mat, time, H_mat[method], dec_sets[dec])
                    for time in times ]
        At_B_mats = [ At_mat @ B_mat for At_mat in At_mats ]
        B_At_mats = [ B_mat @ At_mat for At_mat in At_mats ]
        TTC_mats = [ At_B_mat - At_mat * B_init_val
                     for At_B_mat, At_mat in zip(At_B_mats, At_mats) ]
        comm_mats = [ At_B_mat - B_At_mat for At_B_mat, B_At_mat in zip(At_B_mats, B_At_mats) ]
        FPC_mats = [ comm_mat.conj().T @ comm_mat for comm_mat in comm_mats ]

        corrs_TTC_exact[dec][method] \
            = np.array([ expectation_value(TTC_mat) for TTC_mat in TTC_mats ])
        corrs_FPC_exact[dec][method] \
            = np.array([ expectation_value(FPC_mat) for FPC_mat in FPC_mats ])
        corrs_FPC_exact[dec][method] = np.real(corrs_FPC_exact[dec][method])

    print(time()-start)

In [ ]:
##########################################################################################
# full simulation methods
##########################################################################################

init_state = { method : init_state_X for method in methods }
dec_mat = { method : None for method in methods }

for method in methods:
    if N < 10**4 or method in [ NAT, OAT ]: continue
    init_state[method] = init_state_nZ
    dec_mat[method] = mat_X_to_nZ_pzm
    h_pzm[method] = vec_zxy_to_pzm(h_zxy[method], init_state_X)
    A_pzm[method] = vec_zxy_to_pzm(A_zxy, init_state_X)
    B_pzm[method] = vec_zxy_to_pzm(B_zxy, init_state_X)

def compute_deriv_op_vecs(op, method, dec):
    return get_deriv_op_vec(order_cap, N, init_state[method], h_pzm[method],
                            dec_rates = dec_rates[dec], dec_mat = dec_mat[method],
                            deriv_ops = list(op.keys()), remove_irrelevant_ops = False)

def save_vec(vec, save_file):
    with open(save_file, "w") as f:
        for key in vec.keys():
            f.write(f"key {key}\n")
            for op, val in vec[key].items():
                f.write(f"{op} {val}\n")

def load_vec(load_file):
    vec = {}
    with open(load_file, "r") as f:
        for line in f:
            line_items = line.split()
            if line_items[0] == "key":
                key = eval(" ".join(line_items[1:]))
                vec[key] = {}
            else:
                op = eval(" ".join(line_items[:-1]))
                val = eval(line_items[-1])
                vec[key][op] = val
    return vec

def get_vec(base_name, method, dec, print_text = None):
    if print_text is not None: print(print_text)

    if base_name == vecs_A_str:
        if os.path.isfile(save_file_name(vecs_A_str, method, dec)):
            return load_vec(save_file_name(vecs_A_str, method, dec))
        else:
            vec = compute_deriv_op_vecs(A_pzm[method], method, dec)
            save_vec(vec, save_file_name(vecs_A_str, method, dec))
            return vec

    elif base_name == vecs_At_B_str:
        if os.path.isfile(save_file_name(vecs_At_B_str, method, dec)):
            return load_vec(save_file_name(vecs_At_B_str, method, dec))
        else:
            vec = sandwich_deriv_op_vec(vecs_A[dec][method], append_op = B_pzm[method])
            save_vec(vec, save_file_name(vecs_At_B_str, method, dec))
            return vec

    elif base_name == vecs_B_At_str:
        if os.path.isfile(save_file_name(vecs_B_At_str, method, dec)):
            return load_vec(save_file_name(vecs_B_At_str, method, dec))
        else:
            vec = sandwich_deriv_op_vec(vecs_A[dec][method], prepend_op = B_pzm[method])
            save_vec(vec, save_file_name(vecs_B_At_str, method, dec))
            return vec

    elif base_name == vecs_FPC_str:
        if os.path.isfile(save_file_name(vecs_FPC_str, method, dec)):
            return load_vec(save_file_name(vecs_FPC_str, method, dec))
        else:
            vecs_At_B_comm = add_deriv_op_vecs(vecs_At_B[dec][method],
                                               vecs_B_At[dec][method], factor_rht = -1)
            vec = multiply_deriv_op_vecs(vecs_At_B_comm, vecs_At_B_comm, dag_lft = True)
            save_vec(vec, save_file_name(vecs_FPC_str, method, dec))
            return vec

    else:
        print("basename for vector not recognized:",base_name)
        return None

def get_vecs_parallel(base_name):
    results = {}
    pool = mp.Pool(processes = mp.cpu_count() - free_cores)
    for method, dec in itertools.product(methods, dec_rates):
        args = ( base_name, method, dec, f"{method} {dec} {base_name}" )
        results[method,dec] = pool.apply_async(get_vec, args = args)
    pool.close()
    pool.join()

    vecs = { dec : {} for dec in dec_rates }
    for method, dec in itertools.product(methods, dec_rates):
        vecs[dec][method] = results[method,dec].get()
    return vecs

In [ ]:
start = time()

vecs_A = get_vecs_parallel(vecs_A_str)
vecs_At_B = get_vecs_parallel(vecs_At_B_str)
vecs_B_At = get_vecs_parallel(vecs_B_At_str)

for method, dec in itertools.product(methods, dec_rates):
    deriv_vals_A = deriv_op_vec_to_vals(vecs_A[dec][method], N, init_state[method])
    deriv_vals_At_B = deriv_op_vec_to_vals(vecs_At_B[dec][method], N, init_state[method])

    all_corrs_A = deriv_vals_to_correlators(deriv_vals_A, times)
    all_corrs_At_B = deriv_vals_to_correlators(deriv_vals_At_B, times)

    corrs_A[dec][method] \
        = sum( A_pzm[method][key] * corrs for key, corrs in all_corrs_A.items() )
    corrs_At_B[dec][method] \
        = sum( A_pzm[method][key] * corrs for key, corrs in all_corrs_At_B.items() )

    corrs_TTC[dec][method] = corrs_At_B[dec][method] - corrs_A[dec][method] * B_init_val

print(time()-start)

In [ ]:
start = time()

vecs_FPC = get_vecs_parallel(vecs_FPC_str)

for method, dec in itertools.product(methods, dec_rates):
    deriv_vals_FPC = deriv_op_vec_to_vals(vecs_FPC[dec][method], N, init_state[method])
    all_corrs_FPC = deriv_vals_to_correlators(deriv_vals_FPC, times)
    corrs_FPC[dec][method] \
        = sum( np.real( A_pzm[method][key[0]] * np.conj(A_pzm[method][key[1]]) * corrs )
               for key, corrs in all_corrs_FPC.items() )

print(time()-start)

In [ ]:
max_plot_time = 7
fontsize = 8

figsize_complex = (3,2.6)
figsize_real = (3,2.2)
mag_phs_ratio = 2.5

mag = "mag" # key for correlator magnitude
phs = "phs" # key for correlator phase

params = { "text.usetex" : True,
           "font.size" : fontsize,
           "axes.titlesize" : fontsize,
           "axes.labelsize" : fontsize,
           "legend.fontsize" : fontsize }
plt.rcParams.update(params)
plot_idx = N * times <= max_plot_time


colors = { OAT : "#1E4977", TAT : "#E15759", TNT : "#FFAE4B" }
dashes_list = [ (1,0), (4.5,1.5) ]
linestyles = [ "-", "--", ":", "-." ]

def make_legends(axis):
    color_handles = [ Line2D([0], [0], color = colors[method]) for method in methods ]
    color_legend = axis.legend(color_handles, methods)

    line_handles = [ Line2D([0], [0], dashes = dashes, color = "k")
                     for dashes in dashes_list ]
    line_labels = [ dec_labels[dec] for dec in dec_rates ]
    line_legend = axis.legend(line_handles, line_labels, loc = "lower right")

    axis.add_artist(color_legend)
    axis.add_artist(line_legend)

def get_fit(times, values, method):
    x_0 = values[0]
    if method == OAT:
        def f(x,a,b):
            return x_0 + a*N*x + (b*N*x)**2
    else:
        def f(x,a,b): return x_0 + a * ( np.exp(b*N*x) - 1 )
    return curve_fit(f, times, values)

def plot_complex(corrs, scale, filename, ylabels, phs_yticks = None, fit = False):
    plt.figure(figsize = figsize_complex)
    grid = gridspec.GridSpec(2, 1, height_ratios = [ mag_phs_ratio, 1 ])
    axis = { mag : plt.subplot(grid[0]),
             phs : plt.subplot(grid[1]) }

    for method in methods:
        for dec, dashes in zip(dec_rates.keys(), dashes_list):
            corr = corrs[dec]
            axis[mag].semilogy(N*times[plot_idx], abs(corr[method][plot_idx])/scale,
                               dashes = dashes, color = colors[method])
            axis[phs].plot(N*times[plot_idx], np.angle(corr[method][plot_idx])/np.pi,
                           dashes = dashes, color = colors[method])

            if not fit or dec != dec_0: continue
            fit_values = abs(corrs[dec_0][method])/scale
            popt, pcov = get_fit(times[plot_idx], fit_values[plot_idx], method)
            print(method,dec)
            print(popt)
            print(np.amax(pcov))
            print()

    axis[phs].set_xlabel(r"Time ($N\chi t$)")
    plt.setp(axis[mag].get_xticklabels(), visible = False)

    for val in [ mag, phs ]:
        axis[val].set_ylabel(ylabels[val])

    if phs_yticks is not None:
        axis[phs].set_yticks(phs_yticks)

    make_legends(axis[mag])
    plt.tight_layout()
    plt.savefig(filename)

def plot_real(corrs, scale, filename, ylabel, ymin = None, fit = False):
    plt.figure(figsize = figsize_real)

    for method in methods:
        for dec, dashes in zip(dec_rates.keys(), dashes_list):
            corr = corrs[dec]
            plt.semilogy(N*times[plot_idx], corr[method][plot_idx]/scale,
                         dashes = dashes, color = colors[method])

            if not fit or dec != dec_0: continue
            fit_values = corrs[dec_0][method]/scale
            popt, pcov = get_fit(times[plot_idx], fit_values[plot_idx], method)
            print(method,dec)
            print(popt)
            print(np.amax(pcov))
            print()

    plt.xlabel(r"Time ($N\chi t$)")
    plt.ylabel(ylabel)

    if ymin is not None:
        plt.ylim(ymin,plt.gca().get_ylim()[1])

    make_legends(plt.gca())
    plt.tight_layout()
    plt.savefig(filename)

In [ ]:
%matplotlib agg
TTC_labels = { mag : r"$\left|C(t)\right|$", phs : r"$\phi(t)/\pi$" }
FPC_label = r"$D(t)$"
if N <= exact_simulation_cutoff_size:
    plot_complex(corrs_TTC, S, tmp_dir + "two_time.pdf", TTC_labels)
    plot_complex(corrs_TTC_exact, S, tmp_dir + "two_time_exact.pdf", TTC_labels)
    plot_real(corrs_FPC, S**2, tmp_dir + "FPC.pdf", FPC_label)
    plot_real(corrs_FPC_exact, S**2, tmp_dir + "FPC_exact.pdf", FPC_label)
else:
    plot_complex(corrs_TTC, S, fig_dir + f"two_time_{order_cap}.pdf", TTC_labels,
                 phs_yticks = [ 0, -0.25, -0.5 ], fit = True)
    plot_real(corrs_FPC, S**2, fig_dir + f"FPC_{order_cap}.pdf", FPC_label,
              ymin = 1e-2, fit = True)